In [11]:
import pprint
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Visualization
from pandas.plotting import lag_plot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler


# Statistical tools for time series analysis
from scipy import signal

from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

sns.set_style("whitegrid")
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15)

from pylab import rcParams

In [15]:
data = pd.read_csv(
    'DATA/stocks.csv', 
    usecols = ['Name','Date','Open','Closing_Price','Daily_High','Daily_Low','Volume']
)

# Filtering the dataframe to focus on Air Liquide stocks
air_liquide = data[data['Name'] == 'Air Liquide'].copy()

# Converting the 'Date' column to a datetime object
air_liquide['Date'] = pd.to_datetime(air_liquide['Date'])

# Extracting year and month from the 'Date' column and adding them as separate columns
air_liquide['Year'] = air_liquide['Date'].dt.year
air_liquide['Month'] = air_liquide['Date'].dt.month

# Printing the shape of the 'air_liquide' dataframe
print(f'air_liquide shape: {air_liquide.shape[0]} rows, {air_liquide.shape[1]} columns')

air_liquide shape: 2600 rows, 9 columns


## Handling the Missing DATA

In [17]:
# Check the number of NaN values in each column of the dataframe
air_liquide.isna().sum()

Name               0
Date               0
Open               1
Closing_Price      0
Daily_High         0
Daily_Low          0
Volume           134
Year               0
Month              0
dtype: int64

In [18]:
# Retrieve the index of the missing value in the 'Open' column
index_open_missing = air_liquide[pd.isnull(air_liquide['Open'])].index
print("index of the missing value in the Open column:", list(index_open_missing)[0])

index of the missing value in the Open column: 4538


In [19]:
# Replace the missing value in the 'Open' column with the value from the previous day (backward fill)
air_liquide['Open'] = air_liquide['Open'].fillna(method='bfill')

# Calculate and print the percentage of missing values in the 'Volume' column
print(f"Percentage of missing values in Volume: {round(sum(pd.isnull(air_liquide['Volume']))/air_liquide.shape[0],2)}\n")
print(air_liquide[air_liquide['Volume']==0])

Percentage of missing values in Volume: 0.05

Empty DataFrame
Columns: [Name, Date, Open, Closing_Price, Daily_High, Daily_Low, Volume, Year, Month]
Index: []


In [20]:
# Missing values in the 'Volume' column are replaced by 0
imputer = SimpleImputer(strategy='constant', fill_value=0)
air_liquide_plus = imputer.fit_transform(air_liquide)

# Imputation removed column names; put them back
imputed_air_liquide = pd.DataFrame(air_liquide_plus)

# Create a new dataframe with imputed values
imputed_air_liquide.columns = air_liquide.columns

In [21]:
imputed_air_liquide

,Name,Date,Open,Closing_Price,Daily_High,Daily_Low,Volume,Year,Month
0,Air Liquide,2020-04-02,116.0,112.95,116.0,110.0,331,2020,4
1,Air Liquide,2020-04-01,115.95,114.05,115.95,112.3,480,2020,4
2,Air Liquide,2020-03-31,113.9,114.6,115.35,112.95,784,2020,3
3,Air Liquide,2020-03-30,110.25,112.15,112.7,106.9,485,2020,3
4,Air Liquide,2020-03-27,110.2,107.0,110.2,105.8,525,2020,3
...,...,...,...,...,...,...,...,...,...
2595,Air Liquide,2010-01-08,51.28,51.52,51.52,51.28,64,2010,1
2596,Air Liquide,2010-01-07,50.82,51.34,51.34,50.82,48,2010,1
2597,Air Liquide,2010-01-06,51.49,51.34,51.49,51.34,257,2010,1
2598,Air Liquide,2010-01-05,52.09,51.86,52.09,51.76,"1,363",2010,1


In [25]:
y= imputed_air_liquide.copy()
y.set_index('Date', inplace=True)
print(y)
y.index = pd.to_datetime(y.index)
y

                   Name    Open Closing_Price Daily_High Daily_Low Volume  \
Date                                                                        
2020-04-02  Air Liquide   116.0        112.95      116.0     110.0    331   
2020-04-01  Air Liquide  115.95        114.05     115.95     112.3    480   
2020-03-31  Air Liquide   113.9         114.6     115.35    112.95    784   
2020-03-30  Air Liquide  110.25        112.15      112.7     106.9    485   
2020-03-27  Air Liquide   110.2         107.0      110.2     105.8    525   
...                 ...     ...           ...        ...       ...    ...   
2010-01-08  Air Liquide   51.28         51.52      51.52     51.28     64   
2010-01-07  Air Liquide   50.82         51.34      51.34     50.82     48   
2010-01-06  Air Liquide   51.49         51.34      51.49     51.34    257   
2010-01-05  Air Liquide   52.09         51.86      52.09     51.76  1,363   
2010-01-04  Air Liquide   52.06         52.27      52.27     52.05  2,636   

,Name,Open,Closing_Price,Daily_High,Daily_Low,Volume,Year,Month
Date,,,,,,,,
2020-04-02,Air Liquide,116.0,112.95,116.0,110.0,331,2020,4
2020-04-01,Air Liquide,115.95,114.05,115.95,112.3,480,2020,4
2020-03-31,Air Liquide,113.9,114.6,115.35,112.95,784,2020,3
2020-03-30,Air Liquide,110.25,112.15,112.7,106.9,485,2020,3
2020-03-27,Air Liquide,110.2,107.0,110.2,105.8,525,2020,3
...,...,...,...,...,...,...,...,...
2010-01-08,Air Liquide,51.28,51.52,51.52,51.28,64,2010,1
2010-01-07,Air Liquide,50.82,51.34,51.34,50.82,48,2010,1
2010-01-06,Air Liquide,51.49,51.34,51.49,51.34,257,2010,1


In [38]:
y.to_csv('DATA/preprocessed_data.csv',index= True)

In [25]:
data= pd.read_csv('DATA/preprocessed_data.csv')
data

,Date,Name,Open,Closing_Price,Daily_High,Daily_Low,Volume,Year,Month
0,2020-04-02,Air Liquide,116.00,112.95,116.00,110.00,331,2020,4
1,2020-04-01,Air Liquide,115.95,114.05,115.95,112.30,480,2020,4
2,2020-03-31,Air Liquide,113.90,114.60,115.35,112.95,784,2020,3
3,2020-03-30,Air Liquide,110.25,112.15,112.70,106.90,485,2020,3
4,2020-03-27,Air Liquide,110.20,107.00,110.20,105.80,525,2020,3
...,...,...,...,...,...,...,...,...,...
2595,2010-01-08,Air Liquide,51.28,51.52,51.52,51.28,64,2010,1
2596,2010-01-07,Air Liquide,50.82,51.34,51.34,50.82,48,2010,1
2597,2010-01-06,Air Liquide,51.49,51.34,51.49,51.34,257,2010,1
2598,2010-01-05,Air Liquide,52.09,51.86,52.09,51.76,"1,363",2010,1


In [26]:
reversed_df = data.iloc[::-1].reset_index(drop=True)
reversed_df

,Date,Name,Open,Closing_Price,Daily_High,Daily_Low,Volume,Year,Month
0,2010-01-04,Air Liquide,52.06,52.27,52.27,52.05,"2,636",2010,1
1,2010-01-05,Air Liquide,52.09,51.86,52.09,51.76,"1,363",2010,1
2,2010-01-06,Air Liquide,51.49,51.34,51.49,51.34,257,2010,1
3,2010-01-07,Air Liquide,50.82,51.34,51.34,50.82,48,2010,1
4,2010-01-08,Air Liquide,51.28,51.52,51.52,51.28,64,2010,1
...,...,...,...,...,...,...,...,...,...
2595,2020-03-27,Air Liquide,110.20,107.00,110.20,105.80,525,2020,3
2596,2020-03-30,Air Liquide,110.25,112.15,112.70,106.90,485,2020,3
2597,2020-03-31,Air Liquide,113.90,114.60,115.35,112.95,784,2020,3
2598,2020-04-01,Air Liquide,115.95,114.05,115.95,112.30,480,2020,4


In [27]:
df = reversed_df
df.set_index('Date', inplace=True)  # Set "date" column as index in-place
df = df[['Open']]  # Keep only the "open" column


In [29]:
df

,Open
Date,
2010-01-04,52.06
2010-01-05,52.09
2010-01-06,51.49
2010-01-07,50.82
2010-01-08,51.28
...,...
2020-03-27,110.20
2020-03-30,110.25
2020-03-31,113.90


In [40]:
scaler = MinMaxScaler()

# Fit scaler to the data
scaled_data = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
print(type(scaler))
scaled_data

<class 'sklearn.preprocessing._data.MinMaxScaler'>


,Open
0,0.049579
1,0.049895
2,0.043579
3,0.036526
4,0.041368
...,...
2595,0.661579
2596,0.662105
2597,0.700526
2598,0.722105


In [44]:
train_size = int(0.7 * len(df))
val_size = int(0.15 * len(df))

# Split DataFrame into train, validation, and test sets
train_df = scaled_data.iloc[:train_size]  # First 70% rows for training
val_df = scaled_data.iloc[train_size:train_size+val_size]  # Next 15% rows for validation
test_df = scaled_data.iloc[train_size+val_size:]  # Remaining 15% rows for testing

# Displaying the split DataFrames


In [39]:
train_df.to_csv('data/train.csv', index=True)
val_df.to_csv('data/val.csv', index=True)
test_df.to_csv('data/test.csv', index=True)